In [1]:
import json;
import pandas as pd;
import scipy;
from scipy import stats;
# from apyori import apriori;
import matplotlib.pyplot as plt;
import numpy as np;
import sklearn
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import tree
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
data_path = 'okcupid_11_25_22.csv'
df = pd.read_csv(data_path)

# drop columns you're not using, convert sex to bool
df.loc[ df['sex'] == 'f', 'sex'] = 0
df.loc[ df['sex'] == 'm', 'sex'] = 1
df = df.filter(items = ['age', 'body_type', 'drinks', 'height', 'smokes', 'education', 'orientation', 'sex'])
# drop rows with too many blank values
df = df[df.isnull().sum(axis=1) < 3]

In [3]:
df.fillna('rather not say', inplace=True)
# drop rows with no height
df = df[df['height'] != 'rather not say']
df = df[df['body_type'] != 'rather not say']

attraction_dict = {'attractive': ['athletic', 'curvy', 'fit','full figured', 'jacked', 'thin'],
                   'unattractive': ['overweight', 'skinny', 'used up', 'a little extra', 'average']}
education_dict = {'some_college': ['dropped out of college/university', 'dropped out of two-year college', 'graduated from two-year college', 
                                   'two-year college', 'working on college/university', 'working on two-year college'],
                  'some_high_school': ['dropped out of high school', 'working on high school'], 
                  'post_secondary': ['graduated from law school', 'graduated from masters program', 'graduated from med school',
                                       'graduated from ph.d program', 'law school', 'masters program', 'med school', 'ph.d program'],
                  'college': ['college/university', 'dropped out of masters program', 'dropped out of med school', 'dropped out of ph.d program',
                              'graduated from college/university', 'dropped out of law school', 'working on law school', 'working on masters program',
                              'working on med school', 'working on ph.d program'],
                  'high_school': ['graduated from high school', 'high school'],
                  'space_camp': ['graduated from space camp', 'dropped out of space camp', 'space camp', 'working on space camp']}
frequency_dict = {'a_lot': ['desperately', 'often', 'very often', 'trying to quit', 'yes'],
                  'never': ['not at all', 'no' ],
                  'sometimes': ['rarely', 'socially', 'sometimes', 'when drinking']}
answer_bank = ['athletic', 'curvy', 'fit','full figured', 'jacked', 'a little extra', 'average', 'thin', 'overweight', 'skinny', 'used up',
               'dropped out of college/university', 'dropped out of two-year college', 'graduated from two-year college', 
               'two-year college', 'working on college/university', 'dropped out of law school', 'working on two-year college', 'dropped out of high school', 'working on high school', 
               'dropped out of high school', 'working on high school', 'graduated from law school', 'graduated from masters program', 'graduated from med school',
               'graduated from ph.d program', 'law school', 'masters program', 'med school', 'ph.d program','college/university', 'dropped out of masters program', 'dropped out of med school', 'dropped out of ph.d program',
               'graduated from college/university', 'graduated from high school', 'high school', 'desperately', 'often', 'very often', 'trying to quit', 'yes', 'not at all', 'no', 'rarely', 'socially', 'sometimes', 'when drinking', 
               'working on ph.d program', 'working on law school', 'working on masters program', 'working on med school']

In [4]:
dict_list = [attraction_dict, frequency_dict, education_dict]
columns_list = ['body_type', 'drinks', 'smokes', 'education']

for c in columns_list:
     for entry in df[c]:
        if 'space camp' in entry:
            df.loc[ df[c] == entry, c] = 'space camp'
        else:
             for d in dict_list:
                for key in d:
                    if entry in d[key]:
                        if entry in answer_bank:
                            df.loc[ df[c] == entry, c] = key
                            answer_bank.remove(entry)
df

,age,body_type,drinks,height,smokes,education,orientation,sex
0,22,unattractive,sometimes,75.0,sometimes,some_college,straight,1
1,35,unattractive,a_lot,70.0,never,space camp,straight,1
2,38,attractive,sometimes,68.0,never,post_secondary,straight,1
3,23,attractive,sometimes,71.0,never,some_college,straight,1
4,29,attractive,sometimes,66.0,never,college,straight,1
...,...,...,...,...,...,...,...,...
59933,32,attractive,sometimes,69.0,never,college,bisexual,1
59938,24,attractive,a_lot,72.0,never,some_college,straight,1
59939,42,unattractive,never,71.0,never,post_secondary,straight,1
59940,27,attractive,sometimes,73.0,a_lot,some_college,straight,1


In [5]:
from sklearn import preprocessing
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
df['body_type']= label_encoder.fit_transform(df['body_type'])
df['body_type'].unique()
df

,age,body_type,drinks,height,smokes,education,orientation,sex
0,22,1,sometimes,75.0,sometimes,some_college,straight,1
1,35,1,a_lot,70.0,never,space camp,straight,1
2,38,0,sometimes,68.0,never,post_secondary,straight,1
3,23,0,sometimes,71.0,never,some_college,straight,1
4,29,0,sometimes,66.0,never,college,straight,1
...,...,...,...,...,...,...,...,...
59933,32,0,sometimes,69.0,never,college,bisexual,1
59938,24,0,a_lot,72.0,never,some_college,straight,1
59939,42,1,never,71.0,never,post_secondary,straight,1
59940,27,0,sometimes,73.0,a_lot,some_college,straight,1


In [6]:
df = pd.get_dummies(data=df, columns=['drinks', 'smokes', 'education', 'orientation'])
df.columns

Index(['age', 'body_type', 'height', 'sex', 'drinks_a_lot', 'drinks_never',
       'drinks_rather not say', 'drinks_sometimes', 'smokes_a_lot',
       'smokes_never', 'smokes_rather not say', 'smokes_sometimes',
       'education_college', 'education_high_school',
       'education_post_secondary', 'education_rather not say',
       'education_some_college', 'education_some_high_school',
       'education_space camp', 'orientation_bisexual', 'orientation_gay',
       'orientation_straight'],
      dtype='object')

In [7]:
# drop rows with no height
df = df[df['height'] != 'rather not say']
X = df.filter(items = ['age', 'drinks', 'height', 'smokes', 'education', 'orientation', 'sex']).values
y = df['body_type'].values
y=y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=14)
df

,age,body_type,height,sex,drinks_a_lot,drinks_never,drinks_rather not say,drinks_sometimes,smokes_a_lot,smokes_never,...,education_college,education_high_school,education_post_secondary,education_rather not say,education_some_college,education_some_high_school,education_space camp,orientation_bisexual,orientation_gay,orientation_straight
0,22,1,75.0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
1,35,1,70.0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
2,38,0,68.0,1,0,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,1
3,23,0,71.0,1,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,1
4,29,0,66.0,1,0,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59933,32,0,69.0,1,0,0,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
59938,24,0,72.0,1,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
59939,42,1,71.0,1,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
59940,27,0,73.0,1,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,1


In [8]:
decision_tree_best = tree.DecisionTreeClassifier() 
decision_tree_best.fit(X_train, y_train)
y_preds = decision_tree_best.predict(X_test)

scoresVC_p = precision_score(y_test, y_preds, average=None)
scoresVC_r = recall_score(y_test, y_preds, average=None) 

print("Decision Tree Precision: ", scoresVC_p)
print("Recall: ", scoresVC_r)

Decision Tree Precision:  [0.63858049 0.44410876]
Recall:  [0.94596182 0.07461929]


In [9]:
lr_best = LogisticRegression() 
lr_best.fit(X_train, y_train)
y_preds = lr_best.predict(X_test)

scoresVC_p_lr = precision_score(y_test, y_preds, average=None)
scoresVC_r_lr = recall_score(y_test, y_preds, average=None) 

print("Logistic Regression Precision: ", scoresVC_p_lr)
print("Recall: ", scoresVC_r_lr)

Logistic Regression Precision:  [0.63358779 0.5       ]
Recall:  [0.99941263 0.00101523]


In [10]:
knn_best = KNeighborsClassifier() 
knn_best.fit(X_train, y_train)
y_preds = knn_best.predict(X_test)

scoresVC_p_knn = precision_score(y_test, y_preds, average=None)
scoresVC_r_knn = recall_score(y_test, y_preds, average=None) 

print("KNN Precision: ", scoresVC_p_knn)
print("Recall: ", scoresVC_r_knn)

KNN Precision:  [0.64324863 0.39529758]
Recall:  [0.75829662 0.27309645]


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
knn_pipe = Pipeline([('mms', MinMaxScaler()),
                     ('knn', KNeighborsClassifier())])
params = [{'knn__n_neighbors': [200, 300, 400],
         'knn__weights': ['uniform', 'distance'],
         'knn__leaf_size': [1, 3, 7, 9, 11, 13, 15, 20]}]
gs_knn = GridSearchCV(knn_pipe,
                      param_grid=params,
                      scoring='accuracy',
                      cv=5)
gs_knn.fit(X_train, y_train)
print(gs_knn.best_params_)

{'knn__leaf_size': 20, 'knn__n_neighbors': 400, 'knn__weights': 'uniform'}


In [12]:
y_preds_gscv = gs_knn.predict(X_test)
scoresVC_p_knn = precision_score(y_test, y_preds_gscv, average=None)
scoresVC_r_knn = recall_score(y_test, y_preds_gscv, average=None) 
print("KNN Precision: ", scoresVC_p_knn)
print("Recall: ", scoresVC_r_knn)

KNN Precision:  [0.6339369 0.5      ]
Recall:  [0.99735683 0.00456853]


In [13]:
params_dt = [{'criterion': ['gini', 'entropy'], 'max_depth': [1, 7, 15, None], 'splitter': ['best', 'random'],
           'min_samples_leaf': [1, 4, 7, 10], 'min_samples_split': [3, 7, 11, 16]}]
gs_dt = GridSearchCV(tree.DecisionTreeClassifier(),
                      param_grid=params_dt,
                      scoring='accuracy',
                      cv=5)
gs_dt.fit(X_train, y_train)

print(gs_dt.best_params_)
y_preds_gscv_dt = gs_dt.predict(X_test)
scoresVC_p_dt = precision_score(y_test, y_preds_gscv_dt, average=None)
scoresVC_r_dt = recall_score(y_test, y_preds_gscv_dt, average=None) 
print("Decision Tree GSCV Precision: ", scoresVC_p_dt)
print("Recall: ", scoresVC_r_dt)

{'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 3, 'splitter': 'best'}
Decision Tree GSCV Precision:  [0.63348837 0.        ]
Recall:  [1. 0.]


C:\Users\13369\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
one = 0 
two = 0
zero = 0
class_appearances = [zero, one, two]
for e in y_test: 
    class_appearances[e] += 1
print(class_appearances)

[3405, 1970, 0]


In [15]:
#KNN Precision:  [0.54811412 0.4503632  0.        ]
#Recall:  [0.78334485 0.24833111 0.        ]
#KNN Precision:  [0.55060241 0.45142857 0.        ]
#Recall:  [0.78956462 0.24610592 0.        ]
#KNN Precision:  [0.54511199 0.46153846 0.        ]
#Recall:  [0.89979267 0.12283044 0.        ]
#KNN Precision:  [0.54628317 0.49315068 0.        ]
#Recall:  [0.93192813 0.09612817 0.        ]

'''
{'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 7, 'splitter': 'random'}
Decision Tree GSCV Precision:  [0.54630004 0.48951049 0.        ]
Recall:  [0.93365584 0.09345794 0.        ]


'''

"\n{'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 7, 'splitter': 'random'}\nDecision Tree GSCV Precision:  [0.54630004 0.48951049 0.        ]\nRecall:  [0.93365584 0.09345794 0.        ]\n\n\n"

In [21]:
from sklearn.ensemble import VotingClassifier
est_vec = [('a', decision_tree_best), ('b', lr_best), ('c', knn_best), ('d', gs_knn), ('e', gs_dt)]
eclf = VotingClassifier(estimators=est_vec)
eclf.fit(X_train, y_train)
y_preds_eclf = eclf.predict(X_test)
scoresVC_p_eclf = precision_score(y_test, y_preds_eclf, average=None)
scoresVC_r_eclf = recall_score(y_test, y_preds_eclf, average=None) 
print("Voting GSCV Precision: ", scoresVC_p_eclf)
print("Recall: ", scoresVC_r_eclf)

Voting GSCV Precision:  [0.63350102 0.375     ]
Recall:  [0.99853157 0.00152284]


In [24]:
bc = BaggingClassifier(base_estimator=gs_dt, n_estimators=100)
bc.fit(X_train, y_train)

BaggingClassifier(base_estimator=GridSearchCV(cv=5,
                                              estimator=DecisionTreeClassifier(),
                                              param_grid=[{'criterion': ['gini',
                                                                         'entropy'],
                                                           'max_depth': [1, 7,
                                                                         15,
                                                                         None],
                                                           'min_samples_leaf': [1,
                                                                                4,
                                                                                7,
                                                                                10],
                                                           'min_samples_split': [3,
                                              

In [25]:
y_preds_bm = bc.predict(X_test)
scores_bc_p = precision_score(y_test, y_preds_bm, average=None)
scores_bc_r = recall_score(y_test, y_preds_bm, average=None) 
print("Bagging GSCV Precision: ", scores_bc_p)
print("Recall: ", scores_bc_r)

Bagging GSCV Precision:  [0.63921413 0.45238095]
Recall:  [0.94596182 0.07715736]


In [29]:
print("F1 Scores: ", (2*scores_bc_p*scores_bc_r)/(scores_bc_r+scores_bc_p))

F1 Scores:  [0.76290857 0.13183001]
